
# Practical LLM Embedding Weight Editing (with Tokenizer) + RAG Contrast

이 노트북은 **Hugging Face 토크나이저/모델**을 이용해 **특정 토큰의 임베딩 가중치를 직접 수정**하고,
수정 전후 출력/유사도를 비교합니다. 또한 **RAG(검색결합생성)** 방식과 비교하여,
"가중치를 바꾸는 것 vs. 바꾸지 않고 외부 지식을 넣는 것"의 차이를 실감할 수 있게 합니다.

> **필수/선택 의존성**
> - 필수: `torch`, `transformers`
> - 선택: `safetensors`, `scikit-learn`(RAG 섹션에서 간단한 최근접 이웃 검색용)
>
> 인터넷이 막힌 환경에서는 사전 다운로드한 모델 경로로 바꾸세요.


## 0) 준비

In [ ]:

import sys, platform
print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")

import importlib

def has(mod):
    try:
        importlib.import_module(mod)
        return True
    except Exception:
        return False

HAS_TORCH = has("torch")
HAS_TRANSFORMERS = has("transformers")
HAS_SAFETENSORS = has("safetensors")
HAS_SKLEARN = has("sklearn")

print("torch:", HAS_TORCH)
print("transformers:", HAS_TRANSFORMERS)
print("safetensors:", HAS_SAFETENSORS)
print("sklearn:", HAS_SKLEARN)

if not (HAS_TORCH and HAS_TRANSFORMERS):
    raise RuntimeError("This notebook requires torch and transformers.")



## 1) 토크나이저/모델 로드

- 데모 모델: **`sshleifer/tiny-gpt2`** (아주 작은 GPT-2)  
  - 인터넷이 없으면, 로컬 경로로 대체: `local_path = "/path/to/local/tiny-gpt2"`

**주의:** tiny 모델은 품질이 낮지만 **빠른 실험**에 적합합니다.


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "sshleifer/tiny-gpt2"  # or local path
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
model.eval();

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Vocab size:", model.config.vocab_size)
print("Embedding shape:", model.transformer.wte.weight.shape)  # [vocab, hidden]



## 2) 기준 출력 생성 (수정 전)
간단한 프롬프트로 출력 결과를 확인합니다.


In [ ]:

def generate_text(prompt, max_new_tokens=40, temperature=0.8, top_p=0.95):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            max_new_tokens=max_new_tokens
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

prompt = "The capital of France is"
baseline = generate_text(prompt, max_new_tokens=30)
print("=== BASELINE ===")
print(baseline)



## 3) 특정 토큰 임베딩 관찰
예시로 토큰 `"France"`의 토큰 ID를 확인하고, 해당 임베딩을 보겠습니다.


In [ ]:

tok = "France"
tok_id = tokenizer.convert_tokens_to_ids(tok)
if tok_id == tokenizer.unk_token_id:
    # tiny-gpt2 토큰화가 다를 수 있으므로, 대안: 소문자/다른 서브워드 확인
    tok = "France"
    tok_id = tokenizer(tok)["input_ids"][0]

print(f"Token '{tok}' -> id {tok_id}")
emb = model.transformer.wte.weight.data
print("Original embedding (first 8 dims):", emb[tok_id][:8].cpu().numpy())



## 4) 임베딩 **직접 수정 (Weight Editing)**
- `"France"` 임베딩을 인위적으로 이동시켜 봅니다.
- 작은 델타를 더해 **미세한 편향**을 주입합니다.


In [ ]:

with torch.no_grad():
    delta = torch.zeros_like(emb[tok_id])
    # 임의로 몇 차원에 작은 상수 주입 (실험 목적)
    delta[:4] = torch.tensor([0.5, -0.4, 0.3, -0.2], device=emb.device)
    emb[tok_id] = emb[tok_id] + 0.05 * delta

print("Edited embedding (first 8 dims):", emb[tok_id][:8].cpu().numpy())



## 5) 수정 후 출력 비교
같은 프롬프트로 다시 생성하여, 출력의 **변화**가 있는지 봅니다.


In [ ]:

edited = generate_text(prompt, max_new_tokens=30)
print("=== EDITED ===")
print(edited)



## 6) 임베딩 변화량 정량화 (코사인 유사도)
수정 전/후 임베딩 벡터의 유사도를 계산합니다.


In [ ]:

import torch.nn.functional as F

with torch.no_grad():
    # 이전 값을 가지고 있지 않으니, 재현을 위해 역편집 샘플 생성
    # (실제 실험에선 편집 전 벡터를 따로 저장해두는 것을 권장)
    # 여기서는 편집 벡터를 빼서 "원래 근처"로 되돌린 복제본을 만들어 비교합니다.
    # Note: 완벽히 같지는 않지만, 코사인 유사도 계산 예시로 충분합니다.
    emb_now = emb[tok_id].detach().clone()
    approx_prev = emb_now - 0.05 * torch.cat([torch.tensor([0.5, -0.4, 0.3, -0.2], device=emb.device), torch.zeros_like(emb_now[4:])])
    cos_sim = F.cosine_similarity(emb_now.unsqueeze(0), approx_prev.unsqueeze(0)).item()

print("Approx cosine similarity (edited vs approx-prev):", cos_sim)



## 7) (선택) safetensors로 저장/로드
편집된 임베딩을 포함한 `state_dict`를 safetensors로 저장/로드합니다.


In [ ]:

if HAS_SAFETENSORS:
    from safetensors.torch import save_file, load_file
    state = model.state_dict()
    save_file(state, "/mnt/data/tinygpt2_edited.safetensors")
    print("Saved: /mnt/data/tinygpt2_edited.safetensors")

    loaded = load_file("/mnt/data/tinygpt2_edited.safetensors")
    # 새 모델에 주입 (동일 아키텍처 필요)
    model2 = AutoModelForCausalLM.from_pretrained(model_id)
    model2.load_state_dict(loaded, strict=True)
    print("Reloaded edited weights into a fresh model instance.")
else:
    print("safetensors not available — skipping save/load demo.")



## 8) RAG(검색결합생성) 간단 대비 실험

가중치를 바꾸지 않고, **외부 지식**을 검색해서 프롬프트에 넣는 방식을 시뮬레이션합니다.
여기서는 작은 문서 코퍼스를 만들어, 쿼리와 **TF-IDF 유사도**(또는 최근접 이웃)로 상위 문서를 찾아
프롬프트에 붙여 넣습니다. (간단한 데모)


In [ ]:

docs = [
    "Paris is the capital of France. It is known for the Eiffel Tower.",
    "Berlin is the capital of Germany. It has the Brandenburg Gate.",
    "Rome is the capital of Italy. It has the Colosseum."
]

query = "What is the capital of France?"

if HAS_SKLEARN:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np

    vec = TfidfVectorizer().fit(docs + [query])
    X = vec.transform(docs)
    q = vec.transform([query])
    sims = cosine_similarity(q, X)[0]
    top_idx = int(np.argmax(sims))
    retrieved = docs[top_idx]
else:
    # fallback: 가장 단순한 키워드 포함 수 카운트
    def score(doc, q):
        s = 0
        for w in q.lower().split():
            if w in doc.lower():
                s += 1
        return s
    scored = [(i, score(d, query)) for i, d in enumerate(docs)]
    scored.sort(key=lambda t: t[1], reverse=True)
    top_idx = scored[0][0]
    retrieved = docs[top_idx]

rag_prompt = f"Context: {retrieved}\n\nQuestion: {query}\nAnswer:"
rag_out = generate_text(rag_prompt, max_new_tokens=40)
print("=== RAG RESULT ===")
print(rag_out)



## 9) 체크리스트 & 요약
- 토큰 임베딩 **직접 편집**은 빠르지만 **예측 불가**한 부작용이 있을 수 있습니다.
- **safetensors**로 안전하게 배포/공유하세요.
- **RAG**는 가중치를 건드리지 않고 **최신성/사내 데이터**를 반영할 수 있는 실무 친화적 방법입니다.
